In [2]:
import numpy as np
import pandas as pd 
import torch 
from pathlib import Path

path = Path("./us-patent-phrase-to-phrase-matching")
df = pd.read_csv(path/"train.csv")

In [26]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [27]:
df.describe(include="object")

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [28]:
df['input'] = "TEXT 1: " + df.context + "; TEXT 2: " + df.target + "; ANC: " + df.anchor

In [29]:
df.input.head

<bound method NDFrame.head of 0        TEXT 1: A47; TEXT 2: abatement of pollution; A...
1        TEXT 1: A47; TEXT 2: act of abating; ANC: abat...
2        TEXT 1: A47; TEXT 2: active catalyst; ANC: aba...
3        TEXT 1: A47; TEXT 2: eliminating process; ANC:...
4        TEXT 1: A47; TEXT 2: forest region; ANC: abate...
                               ...                        
36468    TEXT 1: B44; TEXT 2: wooden article; ANC: wood...
36469    TEXT 1: B44; TEXT 2: wooden box; ANC: wood art...
36470    TEXT 1: B44; TEXT 2: wooden handle; ANC: wood ...
36471    TEXT 1: B44; TEXT 2: wooden material; ANC: woo...
36472    TEXT 1: B44; TEXT 2: wooden substrate; ANC: wo...
Name: input, Length: 36473, dtype: object>

In [30]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)

In [31]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [3]:
model_nm = 'microsoft/deberta-v3-small'

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [35]:
def tok_func(x): return tokz(x["input"])

In [36]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [37]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT 1: A47; TEXT 2: abatement of pollution; ANC: abatement',
 [1,
  54453,
  376,
  294,
  336,
  5753,
  346,
  54453,
  392,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  294,
  47284,
  2])

In [38]:
tokz.vocab['of']

1580

In [39]:
tok_ds = tok_ds.rename_columns({'score': 'labels'})

In [40]:
eval_df = pd.read_csv(path/'test.csv')

In [41]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [42]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [43]:
from transformers import TrainingArguments,Trainer

In [44]:
bs = 128
epochs = 4 
lr = 8e-5

In [45]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')
    

In [48]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [49]:
trainer.train()

NameError: name 'trainer' is not defined

In [50]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

NameError: name 'trainer' is not defined

In [51]:
submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False) 


NameError: name 'datasets' is not defined